In [1]:
import tensorflow as tf

In [2]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1
input_size = 784
output_size = 10

In [4]:
# tf Graph Input
x = tf.placeholder(tf.float32, [None, input_size]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, output_size]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.random_normal([input_size,output_size],mean=0.0,stddev=1.0, dtype=tf.float32))
b = tf.Variable(tf.random_normal([output_size],mean=0.0,stddev=1.0, dtype=tf.float32))

In [43]:
# Construct model
logits = tf.matmul(x, W) + b ##this is linear regression
pred = tf.nn.softmax(logits) # Softmax , this is logistic regression

# Minimize error mean square error for linear regression
cost_linear = tf.reduce_mean(tf.reduce_sum(tf.square(logits-y),reduction_indices = 1))
cost_logistic = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
#cost_logistic = tf.reduce_mean(y*tf.log(pred)+ (1 -y) *tf.log(1 -pred))

# Gradient Descent
optimizer_linear = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_linear)
optimizer_logistic = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_logistic)

#cross entropy error
cost_entropy_logistic = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
#adamoptimizer
adam_optimizer =  tf.train.AdamOptimizer(learning_rate).minimize(cost_entropy_logistic) 

In [57]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [58]:
# no of correct prediction
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [59]:
sess = tf.Session()
# Run the initializer
sess.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop) and cost op (to get loss value)
        _, c = sess.run([optimizer_logistic, cost_logistic], feed_dict={x: batch_xs,
                                                          y: batch_ys})
        # Compute average loss
        avg_cost += c / total_batch
    # Display logs per epoch step
    if (epoch+1) % display_step == 0:
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
        acc = sess.run(accuracy,feed_dict = {x: batch_xs, y: batch_ys})
        print (acc)
        ##print("Accuracy:", acc.eval(sesion = sess))

print("Optimization Finished!")

Epoch: 0001 cost= 6.939578774
0.31
Epoch: 0002 cost= 3.913121308
0.54
Epoch: 0003 cost= 2.879443146
0.51
Epoch: 0004 cost= 2.353552332
0.6
Epoch: 0005 cost= 2.034220595
0.65
Epoch: 0006 cost= 1.817293562
0.7
Epoch: 0007 cost= 1.659587062
0.77
Epoch: 0008 cost= 1.538611289
0.74
Epoch: 0009 cost= 1.442500048
0.71
Epoch: 0010 cost= 1.363818900
0.8
Epoch: 0011 cost= 1.298142333
0.72
Epoch: 0012 cost= 1.242008838
0.74
Epoch: 0013 cost= 1.193702089
0.76
Epoch: 0014 cost= 1.151160456
0.79
Epoch: 0015 cost= 1.113435320
0.8
Epoch: 0016 cost= 1.079794331
0.82
Epoch: 0017 cost= 1.049412942
0.81
Epoch: 0018 cost= 1.021899730
0.81
Epoch: 0019 cost= 0.996942834
0.73
Epoch: 0020 cost= 0.973858736
0.8
Epoch: 0021 cost= 0.952789680
0.84
Epoch: 0022 cost= 0.933427631
0.82
Epoch: 0023 cost= 0.915389391
0.83
Epoch: 0024 cost= 0.898567218
0.85
Epoch: 0025 cost= 0.882902654
0.81
Optimization Finished!


In [60]:
prd = sess.run(pred,feed_dict = {x: mnist.test.images, y: mnist.test.labels})
print (prd[0])

[  1.17765325e-04   7.26122070e-13   5.02141884e-09   2.67057794e-06
   1.02045317e-09   6.82574068e-08   7.23278384e-14   9.99840856e-01
   2.13602962e-05   1.73008830e-05]


In [61]:
prd1 = tf.reduce_sum(prd,reduction_indices = 1)

In [62]:
prd1 = prd1.eval(session = sess)
print (prd1[0])

1.0


In [56]:
prd1

array([ 1.        ,  0.99999988,  1.        , ...,  1.        ,
        1.        ,  1.00000012], dtype=float32)

In [22]:
acc1 = sess.run(accuracy,feed_dict = {x: mnist.test.images, y: mnist.test.labels})

In [23]:
print ("accuracy on test set: ",acc1)

accuracy on test set:  0.8294
